## Hausdorff Distance Calculation

# [Link to my Youtube Video Explaining this whole Notebook](https://www.youtube.com/watch?v=vGLazsy0ekQ&list=PLxqBkZuBynVRMORlFw95iNTp9aZzmmz4Y&index=11)

[![Imgur](https://imgur.com/Z9P2nHm.png)](https://www.youtube.com/watch?v=vGLazsy0ekQ&list=PLxqBkZuBynVRMORlFw95iNTp9aZzmmz4Y&index=11)


---

![Imgur](https://imgur.com/dbMgra3.png)

## The Hausdorff distance is defined as the maximum of all minimum distances between two curves. 

### The Hausdorff distance is the longest distance you can be forced to travel by an adversary who chooses a point in one of the two sets, from where you then must travel to the other set. In other words, it is the greatest of all the distances from a point in one set to the closest point in the other set. - [Source Wikipedia](https://en.wikipedia.org/wiki/Hausdorff_distance)

The Hausdorff distance measures the difference between two subsets of a metric space. Intuitively, a metric space is just some set with a built-in distance function;

### Hausdorff distance is the greatest of all the distances from a point in one set to the closest point in the other set.

In [1]:
import numpy as np
from hausdorff import hausdorff_distance

In [2]:
# two random 2D arrays (second dimension must match)
np.random.seed(0)

X = np.random.random((1000,100))
Y = np.random.random((5000,100))

print(X.shape)
print(Y.shape)

(1000, 100)
(5000, 100)


In [3]:
# Test computation of Hausdorff distance with different base distances
print(f"Hausdorff distance test: {hausdorff_distance(X, Y, distance='manhattan')}")
print(f"Hausdorff distance test: {hausdorff_distance(X, Y, distance='euclidean')}")
print(f"Hausdorff distance test: {hausdorff_distance(X, Y, distance='chebyshev')}")
print(f"Hausdorff distance test: {hausdorff_distance(X, Y, distance='cosine')}")

Hausdorff distance test: 29.195107161831146
Hausdorff distance test: 3.668161325513714
Hausdorff distance test: 0.8289005532480906
Hausdorff distance test: 0.21390438319461313


In [4]:
import numba
import numpy as np
from math import sqrt
from hausdorff import hausdorff_distance

@numba.jit(nopython=True, fastmath=True)

def custom_dist(array_x, array_y):
    """
    Custom distance function used in the BiCycleGAN project.
    
    Args:
        array_x (numpy.ndarray): Input array X.
        array_y (numpy.ndarray): Input array Y.
    
    Returns:
        float: Custom distance between array_x and array_y.
    """
    n = array_x.shape[0]
    ret = 0.
    for i in range(n):
        ret += (array_x[i]-array_y[i])**2
    return sqrt(ret)

print(f"Hausdorff custom crazy test: {hausdorff_distance(X, Y, distance=custom_dist)}")

Hausdorff custom crazy test: 3.668161325513714


In [5]:
# a real crazy custom function
@numba.jit(nopython=True, fastmath=True)

def custom_dist(array_x, array_y):
    """
    Custom distance function used in the BiCycleGAN project.
    
    Args:
        array_x (numpy.ndarray): Input array X.
        array_y (numpy.ndarray): Input array Y.
    
    Returns:
        float: Custom distance between array_x and array_y.
    """
    n = array_x.shape[0]
    ret = 0.
    for i in range(n):
        ret += (array_x[i]-array_y[i])**3 / (array_x[i]**2 + array_y[i]**2 + 0.1)
    return ret

print(f"Hausdorff custom crazy test: {hausdorff_distance(X, Y, distance=custom_dist)}")

Hausdorff custom crazy test: 0.08700612847006414


This custom formula incorporates cubic differences between corresponding elements and normalization terms based on the squares of the elements.

The @numba.jit decorator is used to compile the function using the Numba library, which provides just-in-time (JIT) compilation for improved performance. The nopython=True option ensures that the function is compiled to optimized machine code, and fastmath=True allows the compiler to use faster, less accurate math operations.

## Calculating Hausdorff with Scipy and monai

In [8]:
from scipy.spatial.distance import directed_hausdorff
from monai.metrics.utils import get_mask_edges, get_surface_distance

In [9]:
def calculate_hausdorff_scipy(pred, gt, max_dist):
     """
    Calculates the Hausdorff distance using the SciPy library.
    
    Args:
        pred (numpy.ndarray): Predicted binary mask.
        gt (numpy.ndarray): Ground truth binary mask.
        max_dist (float): Maximum distance threshold.
    
    Returns:
        float: Hausdorff distance normalized by the maximum distance.
    """
    if np.all(pred == gt):
        return 0.0
    dist = directed_hausdorff(np.argwhere(pred), np.argwhere(gt))[0]
    
    if dist > max_dist:
        return 1.0
    return dist / max_dist
        

In [30]:
def calculate_hausdorff_monai(pred, gt, max_dist):
    """
    Calculates the Hausdorff distance using the MONAI library.
    
    Args:
        pred (numpy.ndarray): Predicted binary mask.
        gt (numpy.ndarray): Ground truth binary mask.
        max_dist (float): Maximum distance threshold.
    
    Returns:
        float: Hausdorff distance normalized by the maximum distance.
    """
    if np.all(pred == gt):
        return 0.0
    (edges_pred, edges_gt) = get_mask_edges(pred, gt)
    
    surface_distance = get_surface_distance(edges_pred, edges_gt, distance_metric="euclidean")
    
    if surface_distance.shape == (0,):
        return 0.0
    dist = surface_distance.max()
    if dist > max_dist:
        return 1.0
    return dist / max_dist

In [31]:
random_tensor_shape = (128, 128)

max_distance = np.sqrt(random_tensor_shape[0] ** 3 + random_tensor_shape[1] ** 3 ) 

pred = np.random.randint(0, high=2, size=random_tensor_shape)
gt_vector = np.random.randint(0, high=2, size=random_tensor_shape)

print(calculate_hausdorff_scipy(pred, gt_vector, max_distance))
print(calculate_hausdorff_monai(pred, gt_vector, max_distance))




0.0009765625
0.0009765625


### Check Execution time of scipy vs monai for calculating the Hausdorff Distance

In [32]:
import time
start = time.time()

for i in range(100):
    calculate_hausdorff_scipy(pred, gt_vector, max_distance)
    
scipy_time = time.time() - start

print(scipy_time)

2.832296133041382


In [33]:
start = time.time()

for i in range(100):
    calculate_hausdorff_monai(pred, gt_vector, max_distance)
    
monai_time = time.time() - start

print(monai_time)

0.2039167881011963


In [34]:
scipy_time / monai_time

13.889470108933939

## PyTorch Implementation

In [49]:
import torch

def hausdorff_2d_torch(x, y):
    """
    Calculates the Hausdorff distance between two 2D point sets using PyTorch.
    
    Args:
        x (torch.Tensor): Input tensor of shape (batch_size, num_points_x, 2).
        y (torch.Tensor): Input tensor of shape (batch_size, num_points_y, 2).
    
    Returns:
        torch.Tensor: The Hausdorff distance between the point sets of shape (batch_size,).
     """
    x = x.float()
    y = y.float()
    
    distance_tensor = torch.cdist(x, y, p=2)
    
    
    value1 = distance_tensor.min(2)[0].max(1, keepdim=True)[0]
    value2 = distance_tensor.min(1)[1].max(1, keepdim=True)[0]

    value = torch.cat((value1, value2), dim=1)
    
    return value.max(1)[0]

In this code, the hausdorff_2d_torch function calculates the Hausdorff distance between two sets of 2D points using PyTorch.

Next, it computes the pairwise distances between all points in x and y using the torch.cdist function with p=2, which corresponds to the Euclidean distance.

In [50]:
a = torch.randn(2, 3, 4)
b = torch.randn(2, 3, 4)
# print(a)
# print(b)


In [51]:
print('Hausdorff Distance is ', hausdorff_2d_torch(a, b) )

Hausdorff Distance is  tensor([3.0079, 3.5724])
